In [12]:
import numpy as np
import torch

def add_horizontal_noise_rgb(img_tensor, intensity=0.1, stripe_width=1, randomize=False):
    """
    Thêm nhiễu horizontal trắng đen vào ảnh RGB (3 kênh).
    Args:
        img_tensor (torch.Tensor): Ảnh RGB dạng [3, H, W], giá trị [0,1]
        intensity (float): Mức độ nhiễu
        stripe_width (int): Độ dày của các sọc nhiễu
        randomize (bool): Có random cường độ theo dòng không
    Returns:
        torch.Tensor: Ảnh RGB [3, H, W] đã thêm nhiễu trắng đen
    """
    img_np = img_tensor.clone().cpu().numpy()
    c, h, w = img_np.shape
    assert c == 3, "Ảnh đầu vào phải là RGB với 3 kênh"

    # Tạo nhiễu dạng grayscale
    noise = np.zeros((1, h, w), dtype=np.float32)

    for row in range(0, h, stripe_width * 2):
        stripe_noise = np.random.uniform(-intensity, intensity) if randomize else intensity
        noise[:, row:row+stripe_width, :] = stripe_noise

    # Áp dụng nhiễu đều lên cả 3 kênh
    noise = np.repeat(noise, 3, axis=0)  # [1, H, W] -> [3, H, W]

    noisy_img = img_np + noise
    noisy_img = np.clip(noisy_img, 0.0, 1.0)
    return torch.from_numpy(noisy_img).to(img_tensor.device)

def save_tensor_image(noisy_img, output_path):
    img_np = noisy_img.detach().cpu().numpy()  # [C, H, W]
    img_np = np.transpose(img_np, (1, 2, 0))    # → [H, W, C]
    img_np = (img_np * 255).astype(np.uint8)    # [0,1] → [0,255]
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)  # RGB → BGR
    cv2.imwrite(output_path, img_np)
    print(f"Ảnh nhiễu đã được lưu tại {output_path}")

# Giả sử ảnh input là RGB [3, H, W]
from PIL import Image
import torchvision.transforms as T

image_path = "/home/duongnhan/Chon/MWFormer/data/CT/gt/image_s0001_i0021.jpg"
img = Image.open(image_path).convert("RGB")
img_tensor = T.ToTensor()(img)

noisy_img = add_horizontal_noise_rgb(img_tensor, intensity=0.25, stripe_width=8, randomize=True)
output_path = "/home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg"
save_tensor_image(noisy_img, output_path)
print(f"Ảnh nhiễu đã được lưu tại {output_path}")


Ảnh nhiễu đã được lưu tại /home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg
Ảnh nhiễu đã được lưu tại /home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg


(512, 512, 3)

In [9]:
import cv2
import numpy as np
import os
import torch

def add_horizontal_noise(img_tensor, intensity=0.1, stripe_width=1, randomize=False):
    """
    Thêm nhiễu horizontal (dạng sọc ngang) vào ảnh.
    
    Args:
        img_tensor (torch.Tensor): Ảnh input dạng [C, H, W] với giá trị [0, 1]
        intensity (float): Mức độ nhiễu (0.0 - 1.0)
        stripe_width (int): Độ dày của mỗi sọc ngang
        randomize (bool): Nếu True, nhiễu sẽ thay đổi ngẫu nhiên theo từng dòng

    Returns:
        torch.Tensor: Ảnh với nhiễu horizontal
    """
    img_np = img_tensor.clone().cpu().numpy()
    c, h, w = img_np.shape

    noise = np.zeros((c, h, w), dtype=np.float32)

    for row in range(0, h, stripe_width * 2):
        stripe_noise = np.random.uniform(-intensity, intensity) if randomize else intensity
        noise[:, row:row+stripe_width, :] = stripe_noise

    noisy_img = img_np + noise
    noisy_img = np.clip(noisy_img, 0.0, 1.0)
    return torch.from_numpy(noisy_img).to(img_tensor.device)


def add_gaussian_noise_grayscale(image, mean=0, sigma=35):
    # Chuyển ảnh sang kiểu int16 để tránh tràn giá trị
    image = image.astype(np.int16)
    
    # Tạo nhiễu grayscale (chỉ một kênh)
    h, w, c = image.shape
    noise = np.random.normal(mean, sigma, (h, w)).astype(np.int16)  # Chỉ tạo nhiễu cho 1 kênh

    # Áp dụng nhiễu lên cả 3 kênh màu
    noisy_image = np.clip(image + noise[:, :, np.newaxis], 0, 255).astype(np.uint8)

    return noisy_image

# Đọc ảnh
image_path = "/home/duongnhan/Chon/MWFormer/data/CT/gt/image_s0001_i0021.jpg"  # Thay bằng đường dẫn ảnh của bạn
image = cv2.imread(image_path)

if image is None:
    print("Không thể đọc ảnh! Kiểm tra đường dẫn.")
else:
    # Thêm nhiễu Gaussian trắng đen
    # noisy_image = add_horizontal_noise(image)
    noisy_img = add_horizontal_noise(img_tensor, intensity=0.05, stripe_width=2, randomize=True)

    # Lưu ảnh kết quả
    output_path = "/home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg"
    cv2.imwrite(output_path, noisy_image)
    print(f"Ảnh nhiễu đã được lưu tại {output_path}")

    # Hiển thị ảnh gốc và ảnh có nhiễu
    # cv2.imshow("Original Image", image)
    # cv2.imshow("Noisy Image", noisy_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


NameError: name 'img_tensor' is not defined

In [ ]:
import cv2
import numpy as np
import os

def add_horizontal_noise(image, intensity=30, stripe_thickness=1, frequency=20):
    noisy_image = image.copy()
    h, w, c = image.shape

    for i in range(0, h, frequency):
        noise = np.random.randint(-intensity, intensity, (stripe_thickness, w, c), dtype=np.int16)
        start_row = min(i, h - stripe_thickness)
        noisy_image[start_row:start_row + stripe_thickness] = np.clip(
            noisy_image[start_row:start_row + stripe_thickness].astype(np.int16) + noise, 0, 255
        ).astype(np.uint8)

    return noisy_image

def add_gaussian_noise_grayscale(image, mean=0, sigma=35):
    image = image.astype(np.int16)
    h, w, c = image.shape
    noise = np.random.normal(mean, sigma, (h, w)).astype(np.int16)
    noisy_image = np.clip(image + noise[:, :, np.newaxis], 0, 255).astype(np.uint8)
    return noisy_image

def add_noise_to_images(input_dir, output_dir, mean=0, sigma=35):
    os.makedirs(output_dir, exist_ok=True)

    # Hỗ trợ các định dạng ảnh phổ biến
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp')

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(supported_formats):
            img_path = os.path.join(input_dir, filename)
            image = cv2.imread(img_path)

            if image is None:
                print(f"Không thể đọc ảnh: {filename}")
                continue

            noisy_image = add_gaussian_noise_grayscale(image, mean, sigma)

            out_path = os.path.join(output_dir, filename)
            cv2.imwrite(out_path, noisy_image)
            print(f"Đã xử lý: {filename}")

# 🔧 Thay đổi đường dẫn tương ứng với máy bạn:
input_dir = "./input_images"     # Thư mục chứa ảnh gốc
output_dir = "./noisy_images"    # Thư mục để lưu ảnh có nhiễu

add_noise_to_images(input_dir, output_dir)


In [1]:
import numpy as np
from scipy.fftpack import fft2, ifft2, fftshift, ifftshift
from scipy.signal import fftconvolve
from bm3d import gaussian_kernel
from typing import Tuple
import scipy.io
from PIL import Image
import cv2

def get_experiment_kernel(noise_type: str, noise_var: float, sz: tuple = np.array((101, 101))):
    """
    Get kernel for generating noise from specific experiment from the paper.
    :param noise_type: Noise type string, g[0-4](w|)
    :param noise_var: noise variance
    :param sz: size of image, used only for g4 and g4w
    :return: experiment kernel with the l2-norm equal to variance
    """
    # if noiseType == gw / g0
    kernel = np.array([[1]])
    noise_types = ['gw', 'g0', 'g1', 'g2', 'g3', 'g4', 'g1w', 'g2w', 'g3w', 'g4w']
    if noise_type not in noise_types:
        raise ValueError("Noise type must be one of " + str(noise_types))

    if noise_type != "g4" and noise_type != "g4w":
        # Crop this size of kernel when generating,
        # unless pink noise, in which
        # if noiseType == we want to use the full image size
        sz = np.array([101, 101])
    else:
        sz = np.array(sz)

    # Sizes for meshgrids
    sz2 = -(1 - (sz % 2)) * 1 + np.floor(sz / 2)
    sz1 = np.floor(sz / 2)
    uu, vv = np.meshgrid([i for i in range(-int(sz1[0]), int(sz2[0]) + 1)],
                         [i for i in range(-int(sz1[1]), int(sz2[1]) + 1)])

    beta = 0.8

    if noise_type[0:2] == 'g1':
        # Horizontal line
        kernel = np.atleast_2d(16 - abs(np.linspace(1, 31, 31) - 16))

    elif noise_type[0:2] == 'g2':
        # Circular repeating pattern
        scale = 1
        dist = uu ** 2 + vv ** 2
        kernel = np.cos(np.sqrt(dist) / scale) * gaussian_kernel((sz[0], sz[1]), 10)

    elif noise_type[0:2] == 'g3':
        # Diagonal line pattern kernel
        scale = 1
        kernel = np.cos((uu + vv) / scale) * gaussian_kernel((sz[0], sz[1]), 10)

    elif noise_type[0:2] == 'g4':
        # Pink noise
        dist = uu ** 2 + vv ** 2
        n = sz[0] * sz[1]
        spec = (np.sqrt((np.sqrt(n) * 1e-2) / (np.sqrt(dist) + np.sqrt(n) * 1e-2)))
        kernel = fftshift(ifft2(ifftshift(spec)))

    else:  # gw and g0 are white
        beta = 0

    # -- Noise with additional white component --

    if len(noise_type) > 2 and noise_type[2] == 'w':
        kernel = kernel / np.sqrt(np.sum(kernel ** 2))
        kalpha = np.sqrt((1 - beta) + beta * abs(fft2(kernel, (sz[0], sz[1]))) ** 2)
        kernel = fftshift(ifft2(kalpha))

    kernel = np.real(kernel)
    # Correct variance
    kernel = kernel / np.sqrt(np.sum(kernel ** 2)) * np.sqrt(noise_var)

    return kernel

def get_experiment_noise(noise_type: str, noise_var: float, realization: int, sz: tuple)\
        -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Generate noise for experiment with specified kernel, variance, seed and size.
    Return noise and relevant parameters.
    The generated noise is non-circular.
    :param noise_type: Noise type, see get_experiment_kernel for list of accepted types.
    :param noise_var: Noise variance of the resulting noise
    :param realization: Seed for the noise realization
    :param sz: image size -> size of resulting noise
    :return: noise, PSD, and kernel
    """
    np.random.seed(realization)

    # Get pre-specified kernel
    kernel = get_experiment_kernel(noise_type, noise_var, sz)

    # Create noisy image
    half_kernel = np.ceil(np.array(kernel.shape) / 2)

    if len(sz) == 3 and half_kernel.size == 2:
        half_kernel = [half_kernel[0], half_kernel[1], 0]
        kernel = np.atleast_3d(kernel)

    half_kernel = np.array(half_kernel, dtype=int)

    # Crop edges
    noise = fftconvolve(np.random.normal(size=(sz + 2 * half_kernel)), kernel, mode='same')
    noise = np.atleast_3d(noise)[half_kernel[0]:-half_kernel[0], half_kernel[1]:-half_kernel[1], :]

    psd = abs(fft2(kernel, (sz[0], sz[1]), axes=(0, 1))) ** 2 * sz[0] * sz[1]

    return noise, psd, kernel

In [15]:
imagepath = '/home/duongnhan/Chon/MWFormer/data/CT/gt/'
imagename = imagepath + 'image_s0001_i0021.jpg'
y = np.array(Image.open(imagename)) / 255
y.shape

(512, 512, 3)

In [ ]:
import random

imagepath = 'dataset/dataset_structure'
imagename = imagepath + '/ref/1.png'
# Load noise-free image
y = Image.open(imagename).convert('L')
y = np.array(y)
print(y.shape)

noise_list = ['g1', 'g2', 'g3', 'g4']
noise_type = random.choice(noise_list)
print(noise_type)
sigma = 25
noise_var = (sigma/255)**2  # Noise variance 25 std
seed = 8  # seed for pseudorandom noise realization

# Generate noise with given PSD
noise, psd, kernel = get_experiment_noise(noise_type, noise_var, seed, (64, 64))
# N.B.: For the sake of simulating a more realistic acquisition scenario,
# the generated noise is *not* circulant. Therefore there is a slight
# discrepancy between PSD and the actual PSD computed from infinitely many
# realizations of this noise with different seeds.


# Generate noisy image corrupted by additive spatially correlated noise
# with noise power spectrum PSD
# z = np.atleast_3d(y) + np.atleast_3d(noise)
print(y.shape)
print(noise.shape)
z = y + noise
print(z.shape)
z_rang = np.minimum(np.maximum(z, 0), 1)
print(z_rang.shape)
imagepath = 'dataset/brain/train/noisy/'
noisyimagename=imagepath + '33.png'
plt.imsave(noisyimagename, z_rang, cmap='gray') 
#y_est = np.array(Image.open(noisyimagename)) / 255
# Call BM3D With the default settings.
#y_est = bm3d_rgb(z, psd)


# To include refiltering:
# y_est = bm3d_rgb(z, psd, 'refilter');

# For other settings, use BM3DProfile.
# profile = BM3DProfile(); # equivalent to profile = BM3DProfile('np');
# profile.gamma = 6;  # redefine value of gamma parameter
# y_est = bm3d_rgb(z, psd, profile);

# Note: For white noise, you may instead of the PSD
# also pass a standard deviation
# y_est = bm3d_rgb(z, sqrt(noise_var));

# If the different channels have varying PSDs, you can supply a MxNx3 PSD or a list of 3 STDs:
# y_est = bm3d_rgb(z, np.concatenate((psd1, psd2, psd3), 2))
# y_est = bm3d_rgb(z, [sigma1, sigma2, sigma3])


In [ ]:
import random
import numpy as np
from PIL import Image
# from pathlib import Path

# === Đường dẫn ảnh và nơi lưu ===
imagepath = ('/home/duongnhan/Chon/MWFormer/data/CT/gt/image_s0001_i0021.jpg')
savepath = ('/home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg')
# savepath.parent.mkdir(parents=True, exist_ok=True)  # Tạo thư mục nếu chưa có

# === Load ảnh sạch và chuyển thành RGB (512, 512, 3) ===
img = Image.open(imagepath).convert("RGB")
img_np = np.array(img) / 255.0  # Normalize về [0,1]
print("Clean image shape:", img_np.shape)  # (512, 512, 3)
y = Image.open(imagepath).convert('RGB')
y = np.array(y)

# === Chọn loại nhiễu và thiết lập tham số ===
noise_list = ['g1', 'g2', 'g3', 'g4']
# noise_type = random.choice(noise_list)
noise_type = 'g1'
print("Noise type:", noise_type)

sigma = 25
noise_var = (sigma / 255.0) ** 2  # Phương sai nhiễu
seed = 8  # Random seed để tái hiện được

# === Sinh nhiễu có cấu trúc ===
# from your_module import get_experiment_noise  # 🔁 thay bằng đúng nơi bạn import hàm

noise, psd, kernel = get_experiment_noise(
    noise_type=noise_type,
    noise_var=noise_var,
    realization=seed,
    sz=img_np.shape  # (512, 512, 3)
)

print("Noise shape:", noise.shape)

# === Cộng nhiễu vào ảnh gốc và clamp về [0,1] ===
# noisy_img = np.clip(img_np + noise, 0, 1)

print(y.shape)
print(noise.shape)
z = np.atleast_3d(y) + np.atleast_3d(noise)
# z = y + noise
print(z.shape)
z_rang = np.minimum(np.maximum(z, 0), 1)
print(z_rang.shape)

# === Lưu ảnh đã bị nhiễu ===
# Convert ảnh về đúng định dạng [0,255], uint8, BGR
z_uint8 = (z_rang * 255).astype(np.uint8)
z_bgr = cv2.cvtColor(z_uint8, cv2.COLOR_RGB2BGR)

# Lưu ảnh bằng OpenCV
cv2.imwrite(savepath, z_bgr)
print(f"✅ Đã lưu ảnh bị nhiễu tại: {savepath}")

Clean image shape: (512, 512, 3)
Noise type: g1
Noise shape: (512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
✅ Đã lưu ảnh bị nhiễu tại: /home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg


In [20]:
imagepath = '/home/duongnhan/Chon/MWFormer/data/CT/input/'
imagename = imagepath + 'image_s0001_i0021.jpg'
y = np.array(Image.open(imagename)) / 255
y.shape

(512, 512, 3)

In [28]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 4, 2],
       [0, 0, 0, ..., 2, 6, 2],
       [0, 0, 0, ..., 2, 2, 1]], dtype=uint8)

In [29]:
noise

array([[[-0.05992104],
        [-0.0440739 ],
        [-0.03352228],
        ...,
        [ 0.07169429],
        [ 0.08512972],
        [ 0.0993957 ]],

       [[-0.13490802],
        [-0.13744579],
        [-0.13811942],
        ...,
        [ 0.04495801],
        [ 0.03608688],
        [ 0.03162288]],

       [[-0.11379481],
        [-0.11235925],
        [-0.11189128],
        ...,
        [-0.03515404],
        [-0.03593724],
        [-0.03786273]],

       ...,

       [[ 0.19565049],
        [ 0.1951895 ],
        [ 0.18967023],
        ...,
        [-0.18148091],
        [-0.18155723],
        [-0.17228258]],

       [[-0.02704226],
        [-0.04068443],
        [-0.04725832],
        ...,
        [-0.05510122],
        [-0.04289968],
        [-0.03069778]],

       [[ 0.1576523 ],
        [ 0.15468969],
        [ 0.15408522],
        ...,
        [-0.06313315],
        [-0.04371583],
        [-0.02648343]]])

In [10]:
import random
import numpy as np
from PIL import Image
# from pathlib import Path

# === Đường dẫn ảnh và nơi lưu ===
imagepath = ('/home/duongnhan/Chon/MWFormer/data/CT/gt/image_s0001_i0021.jpg')
savepath = ('/home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg')
# savepath.parent.mkdir(parents=True, exist_ok=True)  # Tạo thư mục nếu chưa có

# === Load ảnh sạch và chuyển thành RGB (512, 512, 3) ===
y = Image.open(imagepath).convert('RGB')
y = np.array(y)

# === Chọn loại nhiễu và thiết lập tham số ===
noise_list = ['g1', 'g2', 'g3', 'g4']
# noise_type = random.choice(noise_list)
noise_type = 'g4'
print("Noise type:", noise_type)

sigma = 25
noise_var = (sigma / 255.0) ** 2  # Phương sai nhiễu
seed = 8  # Random seed để tái hiện được

# === Sinh nhiễu có cấu trúc ===

gray_noise, psd, kernel = get_experiment_noise(
    noise_type=noise_type,
    noise_var=noise_var,
    realization=seed,
    # sz=img_np.shape  # (512, 512, 3)
    # sz = (512, 512, 3)
    sz = (512, 512)
)
# noise = np.repeat(gray_noise[:, :, np.newaxis], 3, axis=2)  # shape: (512, 512, 3)
gray_noise = np.squeeze(gray_noise)  # Kết quả: (512, 512)
noise = np.stack([gray_noise] * 3, axis=2)  # (512, 512, 3)

print("Broadcasted noise shape:", gray_noise.shape)
print("Noise shape:", noise.shape)

# === Cộng nhiễu vào ảnh gốc và clamp về [0,1] ===

# Chuyển ảnh sạch về float [0, 1]
y = y.astype(np.float32) / 255.0
z = y + noise
print(z.shape)

z_clamped = np.clip(z, 0, 1)  # Đảm bảo không tràn màu
z_uint8 = (z_clamped * 255).astype(np.uint8)

z_bgr = cv2.cvtColor(z_uint8, cv2.COLOR_RGB2BGR)
cv2.imwrite(savepath, z_bgr)
print(f"✅ Ảnh đã được lưu tại {savepath}")


Noise type: g4
Broadcasted noise shape: (512, 512)
Noise shape: (512, 512, 3)
(512, 512, 3)
✅ Ảnh đã được lưu tại /home/duongnhan/Chon/MWFormer/data/CT/input/image_s0001_i0021.jpg


In [4]:
print("z_uint8.shape =", z_clamped.shape)


z_uint8.shape = (512, 512, 512)


In [13]:
import os
import cv2
import numpy as np
from PIL import Image
from pathlib import Path

def apply_structured_noise_to_folder(
    input_dir,
    output_gt_dir,
    output_noisy_dir,
    output_txt_path,
    noise_types=['g1', 'g4'],
    sigma=25,
    seed=8
):
    image_paths = list(Path(input_dir).glob("*.jpg"))  # you can adjust for .png if needed
    os.makedirs(output_gt_dir, exist_ok=True)
    os.makedirs(output_noisy_dir, exist_ok=True)

    output_txt = []

    for image_path in image_paths:
        base_name = image_path.stem  # Get filename without extension
        img = Image.open(image_path).convert("RGB")
        img_np = np.array(img).astype(np.float32) / 255.0  # Normalize to [0,1]

        for noise_type in noise_types:
            noise, _, _ = get_experiment_noise(
                noise_type=noise_type,
                noise_var=(sigma / 255.0) ** 2,
                realization=seed,
                sz=(img_np.shape[0], img_np.shape[1])
            )

            # Expand grayscale noise to 3 channels
            noise = np.repeat(np.squeeze(noise)[:, :, np.newaxis], 3, axis=2)
            noisy_img = np.clip(img_np + noise, 0, 1)
            noisy_uint8 = (noisy_img * 255).astype(np.uint8)

            # Save clean GT image (once per noise_type)
            gt_filename = f"{base_name}_{noise_type}.jpg"
            gt_path = os.path.join(output_gt_dir, gt_filename)
            cv2.imwrite(gt_path, cv2.cvtColor((img_np * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))

            # Save noisy image
            noisy_filename = f"{base_name}_{noise_type}.jpg"
            noisy_path = os.path.join(output_noisy_dir, noisy_filename)
            cv2.imwrite(noisy_path, cv2.cvtColor(noisy_uint8, cv2.COLOR_RGB2BGR))

            # Write noisy path to output list
            output_txt.append(noisy_path)

    # Save CT.txt with all noisy image paths
    with open(output_txt_path, "w") as f:
        for path in output_txt:
            f.write(path + "\n")

    # return output_txt





In [15]:
apply_structured_noise_to_folder(
    input_dir='/home/duongnhan/Chon/Capstone_project/Backend/PMRF/data/split_data/train',  # thư mục chứa ảnh gốc
    output_gt_dir='./data/CT25_split/train/gt',    # nơi lưu ảnh sạch gốc+_g1/g4
    output_noisy_dir='./data/CT25_split/train/input',  # nơi lưu ảnh nhiễu
    output_txt_path='./data/CT25_split/train/CT.txt',  # nơi ghi đường dẫn ảnh nhiễu
    noise_types=['g1', 'g4'],  # các loại nhiễu muốn áp dụng
    sigma=25,  # độ lệch chuẩn
    seed=8     # để tái lập nhiễu
)
apply_structured_noise_to_folder(
    input_dir='/home/duongnhan/Chon/Capstone_project/Backend/PMRF/data/split_data/test',  # thư mục chứa ảnh gốc
    output_gt_dir='./data/CT25_split/test/gt',    # nơi lưu ảnh sạch gốc+_g1/g4
    output_noisy_dir='./data/CT25_split/test/input',  # nơi lưu ảnh nhiễu
    output_txt_path='./data/CT25_split/test/CT.txt',  # nơi ghi đường dẫn ảnh nhiễu
    noise_types=['g1', 'g4'],  # các loại nhiễu muốn áp dụng
    sigma=25,  # độ lệch chuẩn
    seed=8     # để tái lập nhiễu
)
apply_structured_noise_to_folder(
    input_dir='/home/duongnhan/Chon/Capstone_project/Backend/PMRF/data/split_data/val',  # thư mục chứa ảnh gốc
    output_gt_dir='./data/CT25_split/val/gt',    # nơi lưu ảnh sạch gốc+_g1/g4
    output_noisy_dir='./data/CT25_split/val/input',  # nơi lưu ảnh nhiễu
    output_txt_path='./data/CT25_split/val/CT.txt',  # nơi ghi đường dẫn ảnh nhiễu
    noise_types=['g1', 'g4'],  # các loại nhiễu muốn áp dụng
    sigma=25,  # độ lệch chuẩn
    seed=8     # để tái lập nhiễu
)


In [1]:
import shutil
import os
import torch
from runtest import run_test  
import argparse

# Đường dẫn để lưu file ảnh tạm thời
UPLOAD_FOLDER = "./uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Lấy các tham số để gọi `run_test`
parser = argparse.ArgumentParser(description='Hyper-parameters for network')
parser.add_argument('-val_batch_size', help='Set the validation/test batch size', default=1, type=int)
parser.add_argument('-seed', help='set random seed', default=19, type=int)
parser.add_argument("-restore-from-stylefilter", help='the weights of feature extraction network', type=str, default='/home/duongnhan/Chon/MWFormer/ckpt/lan4/finetune/finetune3/style_best_all')
parser.add_argument('-restore-from-backbone', help='the weights of the image restoration backbone', default='/home/duongnhan/Chon/MWFormer/ckpt/lan4/finetune/finetune3/finetune4_3190000.pth', type=str)     
parser.add_argument('-val_data_dir', default='/home/duongnhan/Chon/Capstone_project/Backend/ModelAI/MWFormer/data/CT_test/', type=str)  # Chỉ đường dẫn đến thư mục upload ảnh
parser.add_argument('-val_filename', default='CT.txt', type=str)  # Sử dụng tên file của ảnh vừa tải lên

args = parser.parse_args()

# Chạy hàm test.py
try:
    psnr, ssim = run_test(args)  # Gọi hàm test với các tham số đã định nghĩa
    print('val_psnr thu: {0:.2f}, val_ssim thu: {1:.4f}'.format(psnr, ssim))
    # return {"message": "Image denoising completed successfully"}
except Exception as e:
    # return {"error": str(e)}
    print("Eror:", str(e))

ImportError: attempted relative import with no known parent package